In [2]:
%pip install networkx==2.5
%pip install  dgl -f https://data.dgl.ai/wheels/repo.html
%pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html
!pip install ipython-autotime
%load_ext autotime



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.6 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 36.5 MB/s eta 0:00:00
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 17.3 MB/s eta 0:00:00
     ━━━━

In [3]:
# get data from drive
from google.colab import drive

drive.mount('/content/gdrive')
drive_path = "/content/gdrive/MyDrive/fyp_data/"

Mounted at /content/gdrive
time: 5min 3s (started: 2023-10-15 06:28:13 +00:00)


Mounted at /content/gdrive


In [13]:
attack_name = "xssdom"
attack_type = "benign"

time: 4.99 ms (started: 2023-10-15 07:40:52 +00:00)


In [5]:
# import required packages
import matplotlib.pyplot as plt
import networkx as nx
import gzip, pickle
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
from dgl.data import DGLDataset
import torch as th
import json
from collections import defaultdict
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from collections import OrderedDict
from dgl.nn.pytorch import GraphConv
import os
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from dgl.dataloading import GraphDataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# set constant seed for consistency across multiple settings
torch.manual_seed(42)
dgl.seed(42)

device_name = torch.device('cpu')
print("Using {}.".format(device_name))

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Using cpu.
time: 9.44 s (started: 2023-10-15 06:33:16 +00:00)


In [6]:
node_types = ['argv', 'block', 'address', 'task', 'process_memory', 'file', 'socket', 'pipe', 'iattr', 'link', 'machine', 'path', 'shm']
edge_types = ['wasAssociatedWith', 'used', 'wasGeneratedBy', 'wasInformedBy', 'wasDerivedFrom', 'wasAttributedTo', 'actedOnBehalfOf']

node_features_count = len(node_types)
edge_features_count = len(edge_types)

targets = np.array(node_types)
nodelabelEnc = LabelEncoder()
new_target = nodelabelEnc.fit_transform(targets)
nodeencoder = OneHotEncoder(sparse_output=False)
nodeencoder.fit(new_target.reshape(-1, 1))
nodeencoder = nodeencoder

targets = np.array(edge_types)
edgelabelEnc = LabelEncoder()
new_target = edgelabelEnc.fit_transform(targets)
edgeencoder = OneHotEncoder(sparse_output=False)
edgeencoder.fit(new_target.reshape(-1, 1))
edgeencoder = edgeencoder


def one_hot_node(data):
  new_target = nodelabelEnc.transform(np.array(data))
  return torch.from_numpy(nodeencoder.transform(new_target.reshape(-1, 1)))

def one_hot_edge(data):
  new_target = edgelabelEnc.transform(np.array(data))
  return torch.from_numpy(edgeencoder.transform(new_target.reshape(-1, 1)))


class ProvenanceDataset(DGLDataset):
    def __init__(self):
      super().__init__(name='provenance')

    def read_graph(self, file_name):
      graph_raw = json.load(open(file_name,"r"))

      list_of_nodes = defaultdict(set)

      node_types = {}
      edge_types = []

      out_edges = []
      in_edges = []

      for key, value in graph_raw.items():
        node1_type, edge_type, node2_type = key.split("-")
        node1_index, node2_index = value
        for i in node1_index:
          node_types[i] = node1_type
          edge_types.append(edge_type)
        for i in node2_index:
          node_types[i] = node2_type


        out_edges = out_edges + node1_index
        in_edges = in_edges + node2_index

      node_types = OrderedDict(sorted(node_types.items()))
      number_of_edges = list(node_types.keys())[-1] + 1

      for i in range(number_of_edges):
        if i not in node_types:
          node_types[i] = 'task'

      out_edges = th.tensor(out_edges)
      in_edges = th.tensor(in_edges)

      g = dgl.graph((out_edges, in_edges))

      g.ndata['attr'] = (one_hot_node(list(node_types.values())))
      g.edata['attr'] = (one_hot_edge(edge_types))

      g = dgl.remove_self_loop(g)
      g = dgl.add_self_loop(g)

      return g

    def process(self):
        self.graphs = []
        self.labels = []

        # attack graph ID
        print(f"{drive_path}{attack_name}_attack")
        for graph_id in tqdm(range(1, 2001)):
          g = self.read_graph(f"{drive_path}{attack_name}_attack/graph{graph_id}.json")
          self.graphs.append(g)
          self.labels.append(1)

        # Convert the label list to tensor for saving.
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

dataset = ProvenanceDataset()
graph, label = dataset[0]
print(graph, label, graph.device, label.device)

/content/gdrive/MyDrive/fyp_data/sqli_attack


100%|██████████| 2000/2000 [10:03<00:00,  3.31it/s]

Graph(num_nodes=1031, num_edges=2294,
      ndata_schemes={'attr': Scheme(shape=(13,), dtype=torch.float64)}
      edata_schemes={'attr': Scheme(shape=(7,), dtype=torch.float64)}) tensor(1) cpu cpu
time: 10min 3s (started: 2023-10-15 06:37:42 +00:00)


In [6]:
print(graph.number_of_nodes())

4006
time: 684 µs (started: 2023-10-14 09:58:45 +00:00)


In [9]:
print(graph.all_edges())

(tensor([   1,   39,   50,  ..., 4003, 4004, 4005]), tensor([   0,   40,   51,  ..., 4003, 4004, 4005]))
time: 5.75 ms (started: 2023-10-14 10:00:15 +00:00)


In [24]:
import json
print(attack_name)

st = set()
for graph_id in tqdm(range(1, 2001)):
  x = f"{drive_path}{attack_name}_attack/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  st.update(keysList)
print(st)
print("attack: " + str(len(st)))


b = set()
for graph_id in tqdm(range(1, 2001)):
  x = f"{drive_path}{attack_name}_benign/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  b.update(keysList)
print(b)
print("benign: " + str(len(b)))

sqli


100%|██████████| 2000/2000 [16:02<00:00,  2.08it/s]


{'file-wasGeneratedBy-task', 'process_memory-wasDerivedFrom-process_memory', 'task-used-block', 'pipe-wasDerivedFrom-pipe', 'file-wasDerivedFrom-pipe', 'process_memory-wasDerivedFrom-link', 'file-wasDerivedFrom-iattr', 'file-wasDerivedFrom-file', 'shm-wasDerivedFrom-shm', 'task-used-link', 'link-wasDerivedFrom-link', 'task-used-process_memory', 'file-wasDerivedFrom-link', 'socket-wasDerivedFrom-socket', 'iattr-wasGeneratedBy-task', 'process_memory-wasDerivedFrom-file', 'file-wasDerivedFrom-socket', 'task-used-pipe', 'task-used-socket', 'pipe-wasGeneratedBy-task', 'task-used-file', 'process_memory-wasDerivedFrom-pipe', 'socket-wasDerivedFrom-task', 'link-wasDerivedFrom-process_memory', 'block-wasGeneratedBy-task', 'pipe-wasDerivedFrom-process_memory', 'socket-wasGeneratedBy-task', 'block-wasDerivedFrom-block', 'process_memory-wasGeneratedBy-task', 'task-wasInformedBy-task', 'file-wasDerivedFrom-process_memory'}
attack: 31


100%|██████████| 2000/2000 [00:10<00:00, 189.18it/s]

{'file-wasGeneratedBy-task', 'process_memory-wasDerivedFrom-process_memory', 'task-used-block', 'pipe-wasDerivedFrom-pipe', 'file-wasDerivedFrom-pipe', 'process_memory-wasDerivedFrom-link', 'file-wasDerivedFrom-iattr', 'file-wasDerivedFrom-file', 'task-used-link', 'link-wasDerivedFrom-link', 'task-used-process_memory', 'file-wasDerivedFrom-link', 'socket-wasDerivedFrom-socket', 'iattr-wasGeneratedBy-task', 'process_memory-wasDerivedFrom-file', 'link-wasGeneratedBy-task', 'task-used-pipe', 'task-used-socket', 'pipe-wasGeneratedBy-task', 'task-used-file', 'process_memory-wasDerivedFrom-pipe', 'socket-wasDerivedFrom-task', 'link-wasDerivedFrom-process_memory', 'block-wasGeneratedBy-task', 'pipe-wasDerivedFrom-process_memory', 'socket-wasGeneratedBy-task', 'block-wasDerivedFrom-block', 'process_memory-wasGeneratedBy-task', 'task-wasInformedBy-task', 'file-wasDerivedFrom-process_memory'}
benign: 30
time: 16min 13s (started: 2023-10-14 17:16:58 +00:00)


In [14]:
import json
print(attack_name)

st = []
for graph_id in tqdm(range(1, 2001)):
  x = f"{drive_path}{attack_name}_attack/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  st.append(len(keysList))
print(st)
print("attack: " + str(sum(st)/len(st)))


b = []
for graph_id in tqdm(range(1, 2001)):
  x = f"{drive_path}{attack_name}_benign/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  b.append(len(keysList))
print(b)
print("benign: " + str(sum(b)/len(b)))

xssdom


100%|██████████| 2000/2000 [08:53<00:00,  3.75it/s]


[17, 18, 21, 18, 16, 17, 17, 17, 17, 18, 18, 18, 17, 14, 21, 15, 19, 18, 19, 16, 18, 19, 16, 17, 21, 22, 23, 17, 21, 23, 18, 22, 23, 19, 23, 21, 18, 23, 18, 18, 21, 19, 21, 21, 23, 17, 21, 23, 22, 19, 19, 23, 19, 23, 23, 21, 23, 19, 22, 20, 19, 23, 19, 21, 22, 18, 21, 21, 19, 21, 21, 20, 18, 17, 21, 19, 22, 18, 18, 18, 21, 17, 22, 19, 18, 19, 17, 19, 19, 20, 19, 17, 19, 23, 17, 16, 18, 22, 19, 21, 19, 19, 21, 21, 21, 19, 17, 17, 18, 23, 20, 19, 16, 23, 19, 21, 19, 23, 19, 19, 17, 17, 19, 19, 23, 21, 21, 19, 21, 23, 21, 17, 22, 21, 21, 21, 18, 19, 23, 17, 23, 23, 23, 21, 23, 19, 19, 19, 23, 23, 23, 19, 19, 25, 20, 23, 21, 21, 19, 19, 21, 19, 18, 23, 19, 17, 23, 21, 19, 21, 19, 21, 18, 20, 21, 22, 21, 21, 19, 21, 22, 19, 16, 21, 21, 22, 21, 17, 21, 23, 20, 22, 20, 18, 18, 21, 19, 22, 23, 23, 19, 21, 19, 21, 21, 22, 17, 23, 17, 21, 21, 22, 21, 19, 20, 18, 21, 21, 21, 19, 19, 22, 21, 19, 21, 20, 23, 21, 21, 16, 21, 20, 21, 22, 21, 18, 21, 19, 23, 21, 21, 21, 21, 19, 19, 16, 21, 19, 21, 22,

100%|██████████| 2000/2000 [08:55<00:00,  3.74it/s]

[18, 14, 15, 21, 11, 14, 17, 12, 18, 14, 19, 18, 19, 18, 18, 18, 22, 18, 15, 13, 17, 16, 15, 19, 18, 18, 17, 21, 17, 19, 19, 19, 17, 16, 21, 17, 14, 14, 15, 17, 20, 16, 17, 13, 16, 18, 15, 16, 18, 17, 16, 14, 19, 19, 17, 14, 16, 17, 18, 18, 18, 17, 18, 15, 17, 16, 14, 14, 14, 14, 15, 16, 17, 17, 18, 17, 17, 20, 19, 17, 17, 19, 23, 19, 16, 17, 19, 19, 18, 19, 20, 17, 16, 15, 19, 21, 19, 17, 19, 16, 19, 19, 17, 17, 14, 16, 16, 17, 17, 16, 17, 17, 16, 17, 17, 19, 14, 17, 17, 14, 21, 17, 17, 17, 17, 16, 17, 17, 17, 17, 19, 17, 14, 15, 17, 17, 19, 12, 20, 19, 17, 16, 16, 19, 17, 17, 20, 17, 18, 17, 17, 17, 16, 19, 19, 17, 18, 19, 19, 19, 17, 18, 17, 19, 19, 19, 17, 16, 17, 15, 17, 23, 17, 19, 14, 16, 19, 21, 19, 17, 17, 16, 17, 17, 19, 19, 17, 19, 23, 17, 18, 17, 19, 17, 16, 16, 16, 19, 19, 17, 17, 16, 18, 19, 17, 16, 17, 16, 17, 17, 16, 17, 16, 15, 15, 22, 18, 19, 17, 19, 17, 16, 18, 16, 14, 17, 18, 18, 17, 19, 19, 17, 15, 23, 16, 16, 18, 17, 18, 17, 17, 21, 18, 19, 17, 18, 17, 19, 19, 18,